In [1]:
# Import libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='darkgrid', palette='hls')

import spacy
nlp = spacy.load("es_core_news_sm")
from gensim.models import Word2Vec

/home/bruno-rg/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./data_processed/task2_depression.csv', index_col=0)
print(df.shape)
df.head(5)

(6853, 5)


,Message,Negatives,Neutrals,Positives,Labels
0,haber ser confuso de momento bien y,0.000,1.000,0.000,0
1,espero ser disfrutar tu estancia en el grupo c...,0.000,1.000,0.000,0
2,ser bueno ese hacer cosa nuevo y que ayudar a ...,0.000,1.000,0.000,0
3,pensar hacer tarea y no yo concentrar,0.306,0.694,0.000,0
4,yo alguno vez él hacer pero este vez tener el ...,0.103,0.794,0.103,0


## Creating embbedings 

### Spacy

In [3]:
def embeddings_with_spacy( messages : pd.core.series.Series):
    embeddings = []
    # Getting the embedding vector for a document
    for message in messages.values:
        doc = nlp(message)
        vector = doc.vector
        embeddings.append(vector)

    embeddings = np.array(embeddings)

    return pd.DataFrame(embeddings)
    #df_w2v['Labels'] = df['Labels'].values

### Gensim

In [4]:
def embeddings_with_gensim( messages : pd.core.series.Series, vector_size : int):

    embeddings = []
    # Separating the words from every message 
    sentences = []
    for message in messages.values:
        sentences.append(message.strip().split(' '))
    
    # Training a model word2vec
    model = Word2Vec(sentences, min_count=1, vector_size = vector_size)
    embeddings = []
    for sentence in sentences:   
        vector = np.zeros(vector_size)
        total = 0 

        for j in range(len(sentence)):
            total += 1
            vector += model.wv[sentence[j]] 

        vector = vector / total
        embeddings.append(vector)

    embeddings = np.array(embeddings)

    return pd.DataFrame(embeddings)

In [5]:
df_embeddings = embeddings_with_spacy(df['Message'])

#df_embeddings = embeddings_with_gensim(df['Message'] , 96)

df_embeddings['Negatives'] = df['Negatives'].values
df_embeddings['Neutrals'] = df['Neutrals'].values
df_embeddings['Positives'] = df['Positives'].values
df_embeddings['Labels'] = df['Labels'].values

df_embeddings.shape

(6853, 104)

In [6]:
df_embeddings.to_csv('./data_processed/embeddings_spacy.csv', encoding='utf-8')